<a href="https://colab.research.google.com/github/gondow/rust-future/blob/main/notebooks/rust_future.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

Rust環境をセットアップする（1分弱ほど時間かかります）
- `export PATH=$HOME/.cargo/bin:$PATH`が効かないのでPythonでやってます
- ジェネレータを使うために nightly にしてます（nightly機能は不安定なので，以下のコードがそのままでは動かなくなる可能性大です．2025年6月には動いてました．）

In [13]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:" + os.environ['PATH']
!rustup install nightly
!rustup default nightly
!rustc --version

info: downloading installer
warn: It looks like you have an existing rustup settings file at:
warn: /root/.rustup/settings.toml
warn: Rustup will install the default toolchain as specified in the settings file,
warn: instead of the one inferred from the default host triple.
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
warn: Updating existing toolchain, profile choice will be ignored
info: syncing channel updates for 'nightly-x86_64-unknown-linux-gnu'
info: default toolchain set to 'nightly-x86_64-unknown-linux-gnu'

  nightly-x86_64-unknown-linux-gnu unchanged - rustc 1.90.0-nightly (bdaba05a9 2025-06-27)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following

ファイル `hello.rs` を作成する

In [ ]:
%%writefile hello.rs
fn main() {
    println!("Hello, world!");
}

Writing hello.rs


`hello.rs` をコンパイルして実行する

In [ ]:
!rustc hello.rs
!./hello

Hello, world!


# Iterator

In [5]:
%%writefile iterator.rs
fn main() {
    for i in 0..3 {
        println!("{}", i);
    }
}

Writing iterator.rs


In [6]:
!rustc iterator.rs
!./iterator

0
1
2


`0..3`はイテレータ．イテレータは`next()`を呼ぶと「次の値」をオプショナル型として返す．

In [1]:
%%writefile generator2.rs
fn main() {
   let mut iter = 0..3;
   println!("{:?}", iter.next()); // Some(0)
   println!("{:?}", iter.next()); // Some(1)
   println!("{:?}", iter.next()); // Some(2)
   println!("{:?}", iter.next()); // None
}

Writing generator2.rs


In [4]:
!rustc generator2.rs
!./generator2

Some(0)
Some(1)
Some(2)
None


自分でもイテレータを定義できる．以下はCounterイテレータの定義例．途中でreturnして値を返して，次にnext()を呼ばれる時，イテレータ構造体の`count`の値が状態として保存されているので，順番にインクリメントされた値が返る．

In [9]:
%%writefile counter.rs
struct Counter {
    count: usize,
}
impl Iterator for Counter {
    type Item = usize;
    fn next(&mut self) -> Option<usize> {
        if self.count < 3 {
            self.count += 1;
            Some(self.count)
        } else {
            None
        }
    }
}
fn main() {
    let mut counter = Counter {count: 0};
    println!("{:?}", counter.next ()); // Some(1)
    println!("{:?}", counter.next ()); // Some(2)
    println!("{:?}", counter.next ()); // Some(3)
    println!("{:?}", counter.next ()); // None
}

Overwriting counter.rs


In [8]:
!rustc counter.rs
!./counter

Some(1)
Some(2)
Some(3)
None


Iteratorの定義は↓こうなっている．
```
pub trait Iterator {
    type Item;
    fn next(&mut self) -> Option<Self::Item>;
    // 他にもたくさん
}
```
詳しくは[公式の説明](https://doc.rust-lang.org/std/iter/trait.Iterator.html)を見てね．

# Generator (Coroutine)

In [14]:
%%writefile generator.rs
#![feature(coroutines, coroutine_trait)]
#![feature(stmt_expr_attributes)]
use std::pin::Pin;
use std::ops::{Coroutine};
fn main() {
    let mut coro = #[coroutine] || {
        println!("Start");
        yield 1;
        println!("Middle");
        yield 2;
        println!("End");
        return 3;
    };
    let mut coro = Pin::new(&mut coro);
    println!("{:?}", coro.as_mut().resume(())); // Yielded(1)
    println!("{:?}", coro.as_mut().resume(())); // Yielded(2)
    println!("{:?}", coro.as_mut().resume(())); // Complete(3)
}

Overwriting generator.rs


In [15]:
!rustc generator.rs
!./generator

Start
Yielded(1)
Middle
Yielded(2)
End
Complete(3)


ジェネレータ（コルーチン）とは
- 状態を保存して途中で一時停止(yield)して値を返し，後から再開(resume)できる関数（コルーチン）
- 上記の例では`yield 1;`で値1を返すと，次に`resume`された時に「`yield 1;`の直後」から実行が再開される
- ジェネレータでは`yield`を使えるが，イテレータでは使えない
- 現在，ジェネレータは非推奨機能であり，`async/await`を使うことが推奨されている（なじぇ！？→自己参照でunsafeになるので`async/await`で抽象化（隠蔽）したいらしい）

ジェネレータとイテレータの主な違い：

| 観点           | イテレータ（`Iterator`）                  | ジェネレータ（`Coroutine` / 旧 `Generator`）                   |   |                                     |
| ------------ | ---------------------------------- | ----------------------------------------------------- | - | ----------------------------------- |
| 定義方法         | `struct` + `impl Iterator for ...` | \`let g =                                             \|\| { yield ...; return ... };\`（クロージャ） |
| トレイト名        | `Iterator`                         | `Coroutine`（以前は `Generator`）                          |   |                                     |
| メソッド名        | `next()`                           | `resume(arg)`                                         |   |                                     |
| 戻り値の型        | `Option<T>`                        | `CoroutineState<Y, R>` (`Yielded(Y)` / `Complete(R)`) |   |                                     |
| 状態の保存        | ユーザーが構造体で管理する                      | Rust ランタイムが自動的に状態（中断点）を保存                             |   |                                     |
| 停止と再開        | 自前で実装（ループ・分岐）                      | `yield` と `resume` による自動停止・再開                         |   |                                     |
| 双方向通信（値の送受信） | 基本不可（値を返すだけ）                       | `resume(arg)` → `yield` で双方向通信可能                      |   |                                     |
| 実装の容易さ       | 明示的なステートマシンを書く必要がある                | `yield` を書くだけで状態機械を自動生成できる                            |   |                                     |
| 安定性          | ✅ 安定版で使える                          | 🚧 Nightly 限定。`#![feature(coroutines)]` 必須            |   |                                     |
| 用途           | イテレーション（for文など）                    | 複雑な制御・状態遷移・非同期処理の構築など                                 |   |                                     |


# Future